In [1]:
import numpy as np
import os
import pyedflib
from utils.EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

import read_edf_files
kernels, chans = 1, 64
label_mapping = {
        1: "Rest",
        2: "Squeeze Both Fists",
        3: "Squeeze Both Feet",
        4: "Squeeze Left Hand",
        5: "Squeeze Right Hand",
    }
num_labels = 5
X, Y = read_edf_files.reader() # use other function to read the edf files

2023-10-03 21:18:29.776735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 21:18:30.013055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-10-03 21:18:30.013073: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-03 21:18:30.842025: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file

FileNotFoundError: [Errno 2] No such file or directory: '/home/kaleb/Documents/eeg_dataset/files/S001'

In [2]:
################################################################
## Process, filter, and epoch the data
# init arrays to train/validate/test. Make split 50/25/25
half = int(len(X) / 2)
quarter = int(half / 2)
three_fourths = half + quarter

X_train = X[:half, :, :]
X_validate = X[half : three_fourths, :, :]
X_test = X[three_fourths:, :, :]

y_train_int = Y[:half]
y_validate_int = Y[half:three_fourths]
y_test_int = Y[three_fourths:]

# convert labels to one-hot encoding
y_train = np_utils.to_categorical(y_train_int-1)
y_validate = np_utils.to_categorical(y_validate_int-1)
y_test = np_utils.to_categorical(y_test_int-1)

# convert data to NHWC (trials, channels, samples, kernels) format
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], kernels)
X_validate = X_validate.reshape(X_validate.shape[0], X_validate.shape[1], X_validate.shape[2], kernels)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], kernels)

print('x_train shape: ', X_train.shape, '\ny_train shape: ', y_train.shape)
################################################################
## Call EEGNet

model = EEGNet(nb_classes=num_labels, Chans=X_train.shape[1], Samples=X_train.shape[2],
               dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16,
                 dropoutType= 'Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during 
# optimization to balance it out. This data is approximately balanced so we 
# don't need to do this, but is shown here for illustration/completeness. 
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1, 4:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
# fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 30, 
#                         verbose = 2, validation_data=(X_validate, y_validate),
#                        callbacks=[checkpointer], class_weight = class_weights)

model.fit(X_train, y_train, batch_size = 16, epochs = 30, 
          verbose = 2, validation_data=(X_validate, y_validate),
          callbacks=[checkpointer], class_weight = class_weights)

# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

###############################################################################
# can alternatively used the weights provided in the repo. If so it should get
# you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
# system.
###############################################################################

# WEIGHTS_PATH = /path/to/EEGNet-8-2-weights.h5 
# model.load_weights(WEIGHTS_PATH)

###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

x_train shape:  (90, 64, 656, 1) 
y_train shape:  (90, 5)
test print statement


2023-10-02 22:14:08.053958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-02 22:14:08.053987: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-02 22:14:08.054005: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kaleb-School): /proc/driver/nvidia/version does not exist
2023-10-02 22:14:08.054282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30

Epoch 1: val_loss improved from inf to 1.56743, saving model to /tmp/checkpoint.h5
6/6 - 3s - loss: 1.6126 - accuracy: 0.3222 - val_loss: 1.5674 - val_accuracy: 0.3556 - 3s/epoch - 434ms/step
Epoch 2/30

Epoch 2: val_loss improved from 1.56743 to 1.53882, saving model to /tmp/checkpoint.h5
6/6 - 1s - loss: 1.4426 - accuracy: 0.5222 - val_loss: 1.5388 - val_accuracy: 0.4667 - 1s/epoch - 217ms/step
Epoch 3/30

Epoch 3: val_loss improved from 1.53882 to 1.51116, saving model to /tmp/checkpoint.h5
6/6 - 1s - loss: 1.3263 - accuracy: 0.5889 - val_loss: 1.5112 - val_accuracy: 0.5111 - 1s/epoch - 221ms/step
Epoch 4/30

Epoch 4: val_loss improved from 1.51116 to 1.48026, saving model to /tmp/checkpoint.h5
6/6 - 1s - loss: 1.2660 - accuracy: 0.6333 - val_loss: 1.4803 - val_accuracy: 0.5333 - 1s/epoch - 219ms/step
Epoch 5/30

Epoch 5: val_loss improved from 1.48026 to 1.45718, saving model to /tmp/checkpoint.h5
6/6 - 1s - loss: 1.1820 - accuracy: 0.6333 - val_loss: 1.4572 - val_accur

2023-10-02 22:14:50.309040: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 42991616 exceeds 10% of free system memory.


In [1]:
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

# Assuming you have a trained model called 'model'
# X: EEG data, y: labels

# Calculate accuracy on the original data
original_accuracy = acc # accuracy_score(Y, model.predict(X).argmax(axis=1))

# Initialize a list to store feature importances
feature_importances = []

# Iterate through each electrode
for electrode_idx in range(64):
    # Create a copy of the original data
    X_permuted = X.copy()
    
    # Permute the values of the current electrode
    X_permuted[:, electrode_idx, :] = shuffle(X_permuted[:, electrode_idx, :])
    print(X_permuted.shape)
    print(X.shape)
    exit()
    
    # Calculate accuracy on the permuted data
    probs       = model.predict(X_permuted)
    preds       = probs.argmax(axis = -1)  
    permuted_accuracy = np.mean(preds == y_test.argmax(axis=-1))
    # permuted_accuracy = accuracy_score(Y, model.predict(X_permuted).argmax(axis=1))
    
    # Calculate feature importance as the drop in accuracy
    feature_importance = original_accuracy - permuted_accuracy
    
    # Append the feature importance score to the list
    feature_importances.append(feature_importance)

# Now 'feature_importances' list contains the importance scores for each electrode
# You can associate these scores with corresponding electrode names/indexes for further analysis


NameError: name 'acc' is not defined